In [1]:
from openbabel import pybel
import torch
import molgrid
import numpy as np
import pandas as pd
import os
import re

import seaborn as sns
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.colors import ListedColormap

ModuleNotFoundError: No module named 'openbabel'

In [ ]:
structs_path = './_data/demo'
datadir = os.listdir(structs_path)
len(datadir)

In [ ]:
sdf_structs = {}
for i in range(0, len(datadir)):
    if datadir[i] == 'index' or datadir[i] == 'readme':
        continue
    struct = os.listdir(structs_path + '/'+ datadir[i])
    key = datadir[i]
    value = list(filter(lambda s : re.search(".*.sdf$", s), struct))
    sdf_structs.update({key : {'archive': value[0], 'cord': None}})

In [ ]:
cords = []
for key, value in sdf_structs.items():
    archive = value['archive']
    f = open(f'{structs_path}/{key}/{archive}', "r")
    sdf_sting = f.read()
    mol = pybel.readstring('sdf',sdf_sting)
    cord = molgrid.CoordinateSet(mol)
    sdf_structs.update({key : {'archive': archive, 'cord': cord}})

In [ ]:
sdf_structs['4eky']

In [ ]:
gmaker = molgrid.GridMaker()
dims = gmaker.grid_dimensions(molgrid.defaultGninaLigandTyper.num_types())

In [ ]:
how_many_times = 0
for key, value in sdf_structs.items():
    if(how_many_times > 2):
        break
    print(f'In Progress {how_many_times + 1}/{len(sdf_structs.items())}')
    gridtensor = torch.zeros(dims,dtype=torch.float32)
    gmaker.forward(value['cord'].center(),value['cord'],gridtensor)

    xs = []
    ys = []
    zs = []
    cs = []

    for i in gridtensor:
        x = []
        y = []
        z = []
        c = []
        count_j = 0
        for j in i:
            count_k = 0
            for k in j:
                count_l = 0
                for l in k:
                    if (float(i[count_j][count_k][count_l]) != 0.0):
                        x.append(count_j)
                        y.append(count_k)
                        z.append(count_l)
                        c.append(float(i[count_j][count_k][count_l]))
                    count_l += 1
                count_k += 1
            count_j += 1
        xs.append(x)
        ys.append(y)
        zs.append(z)
        cs.append(c)
        
    for i in range(0, len(xs)):
        
        if len(xs[i]) == 0:
            continue
        xis = xs[i]
        yps = ys[i]
        zee = zs[i]

        # axes instance
        fig = plt.figure(figsize=(6,6))
        ax = Axes3D(fig, auto_add_to_figure=False)
        fig.add_axes(ax)

        # get colormap from seaborn
        cmap = ListedColormap(sns.color_palette("husl", 256).as_hex())

        # plot
        sc = ax.scatter(xis, yps, zee, s=15, c=cs[i], marker='o', cmap=cmap, alpha=1)
        ax.set_xlabel('X Label')
        ax.set_ylabel('Y Label')
        ax.set_zlabel('Z Label')

        # legend
        plt.legend(*sc.legend_elements(), bbox_to_anchor=(1.05, 1), loc=2)

        # save
        plt.savefig(f'{structs_path}/{key}/{key}_{i}_plot', bbox_inches='tight')
    how_many_times = how_many_times + 1